### IFN680 Sokoban Assignment


The functions and classes defined in this module will be called by a marker script. 
You should complete the functions and classes according to their specified interfaces.

You are not allowed to change the defined interfaces.
That is, changing the formal parameters of a function will break the 
interface and triggers to a fail for the test of your code.
 

In [1]:
# Imports

import search
import sokoban

In [4]:
def my_team():
    '''
    Return the list of the team members of this assignment submission as a list
    of triplet of the form (student_number, first_name, last_name)
    '''
    return [ (11351519, 'William', 'Hung'), (11371188, 'Komei', 'Soda') ]

In [12]:
def taboo_cells(warehouse):
    '''  
    Identify the taboo cells of a warehouse. A cell inside a warehouse is 
    called 'taboo' if whenever a box get pushed on such a cell then the puzzle 
    becomes unsolvable.  
    When determining the taboo cells, you must ignore all the existing boxes, 
    simply consider the walls and the target cells.  
    Use only the following two rules to determine the taboo cells;
     Rule 1: if a cell is a corner inside the warehouse and not a target, 
             then it is a taboo cell.
     Rule 2: all the cells between two corners inside the warehouse along a 
             wall are taboo if none of these cells is a target.
    
    @param warehouse: a Warehouse object

    @return
       A string representing the puzzle with only the wall cells marked with 
       an '#' and the taboo cells marked with an 'X'.  
       The returned string should NOT have marks for the worker, the targets,
       and the boxes.  
    '''
    
    # set constants
    wall_cell = '#'
    target_cell = '.'
    goal_cell = '*'
 
    rows, cols = len(warehouse), len(warehouse[0])
    
    # set the cells in the warehouse
    # wall cells -> '#'
    walls = {(r, c) for r in range(rows) for c in range(cols) if warehouse[r][c] == wall_cell}
    # targets are cells that should be filled with boxes -> '.'
    targets = {(r, c) for r in range(rows) for c in range(cols) if warehouse[r][c] == target_cell}
    # goals are cells that already filled with boxes -> '*'
    goals = {(r, c) for r in range(rows) for c in range(cols) if warehouse[r][c] == goal_cell}
    
    # Rule 1: corner cells
    def is_corner_cell(cell):
        '''
        cells that are in the corner:
        has at least one wall on the right/left
        and at least one wall above/below
        '''
        
        # assign row and column
        r, c = cell
        
        # set the var for number of walls count
        num_td_walls = 0
        num_lr_walls = 0
        
        # count num of top/down wall
        if (warehouse[r+1][c] == wall_cell) or (warehouse[r-1][c] == wall_cell):
            num_td_walls += 1
        # count num of right/left wall
        if (warehouse[r][c+1] == wall_cell) or (warehouse[r][c-1] == wall_cell):
            num_lr_walls += 1
            
        return (num_td_walls >= 1) and (num_lr_walls >= 1) and (warehouse[r][c] != target_cell)
        
    taboo_cells_set = {cell for cell in [(r, c) for r in range(rows) for c in range(cols) if is_corner_cell(cell)]}  
    
    # Rule 2: cell between corners
    # mark cells between two corners along a wall as taboo
    for cell_1 in taboo_cells_set:
        for cell_2 in taboo_cells_set:
            if cell_1 != cell_2:
                r1, c1 = cell_1
                r2, c2 = cell_2
                if r1 == r2: # cells in the same row
                    for c in range(min(c1, c2) + 1, max(c1, c2)):
                        if warehouse[r1][c] != target_cell: # rule 2
                            taboo_cells_set.add((r1, c))
                            
                elif c1 == c2: # cells in the same col
                    for r in range(min(r1, r2) + 1, max(r1, r2)):
                        if warehouse[r][c1] != target_cell:
                            taboo_cells_set.add((r, c1))
                            
    # create the taboo cells string
    # TBD

    return taboo_cells_set
                
            
        
        


In [13]:

class SokobanPuzzle(search.Problem):
    '''
    An instance of the class 'SokobanPuzzle' represents a Sokoban puzzle.
    An instance contains information about the walls, the targets, the boxes
    and the worker.

    Your implementation should be fully compatible with the search functions of 
    the provided module 'search.py'. 
    
    Each instance should have at least the following attributes
    - self.allow_taboo_push
    - self.macro
    
    When self.allow_taboo_push is set to True, the 'actions' function should 
    return all possible legal moves including those that move a box on a taboo 
    cell. If self.allow_taboo_push is set to False, those moves should not be
    included in the returned list of actions.
    
    If self.macro is set True, the 'actions' function should return 
    macro actions. If self.macro is set False, the 'actions' function should 
    return elementary actions.
    
    
    '''
    #
    #         "INSERT YOUR CODE HERE"
    #
    #     Revisit the sliding puzzle and the pancake puzzle for inspiration!
    #
    #     Note that you will need to add several functions to 
    #     complete this class. For example, a 'result' function is needed
    #     to satisfy the interface of 'search.Problem'.

    
    def __init__(self, warehouse, allow_taboo_push=True, macro=True):
        self.warehouse = warehouse
        self.walls = tuple(warehouse.walls)
        self.worker = tuple(warehouse.worker)
        self.boxes = tuple(warehouse.boxes)
        self.target = tuple(warehouse.targets)
        self.allow_taboo_push = allow_taboo_push
        self.macro = macro
        self.initial = (tuple(warehouse.worker),) + tuple(warehouse.boxes)
        self.taboo_cell_sets = taboo_cells(warehouse)

    def actions(self, state):
        """
        Return the list of actions that can be executed in the given state.
        
        As specified in the header comment of this class, the attributes
        'self.allow_taboo_push' and 'self.macro' should be tested to determine
        what type of list of actions is to be returned.
        """
        worker, *boxes = state
        allow_actions = []

        for d_row, d_col, action_str in [(0, -1, 'Left'), (0, 1, 'Right'), (-1, 0, 'Top'), (1, 0, 'Down')]:
            new_worker = (worker[0] + d_row, worker[1] + d_col)

            # new_worker posiiton is not the wall
            if new_worker not in self.walls:  

                # if worker move to the box position
                if new_worker in self.boxes:
                    # set the new_box position
                    new_box = (new_worker[0] + d_row, new_worker[1] + d_col)
                    
                    # if new_box position is not wall/box
                    if new_box not in self.walls and new_box not in self.boxes:

                        # check of the move is allowed or the new box position
                        if (self.allow_taboo_push) or (new_box not in self.taboo_cell_sets):
                            
                            # append the action
                            allow_actions.append(('Push', (d_row, d_col), action_str))

                # if worker is not pushing a box
                else:
                    allow_actions.append(('Move', (d_row, d_col), action_str))

        if self.macro:
            # If macro actions are allowed
            # TODO: Implement macro actions (e.g., pushing a box repeatedly in the same direction)
            pass    

        return allow_actions
    
    def result(self, state, action):
        """
        Return the successor of the given state 
        after executing the given action.
        """
        action_type, (d_row, d_col), _ = action
        worker, *boxes = state

        new_worker = (worker[0] + d_row, worker[1] + d_col)

        if action_type == 'Move':
            new_state = (new_worker,) + tuple(boxes)

        elif action_type == 'Push':
            new_box = (new_worker[0] + d_row, new_worker[1] + d_col)
            index = boxes.index(new_box)
            new_boxes = boxes[:index] + ((new_box[0] + d_row, new_box[1] + d_col),) + boxes[index+1:]
            new_state = (new_worker,) + new_boxes

        return new_state

    def is_goal_state(self, state):
        """
        Return True if the state is a goal state; False otherwise.
        """
        for box in state[1:]:
            if box not in self.target:
                return False
            return True
            
    def h(self, n):
        """
        Return the heuristic value for a given node.
        """
        worker, *boxes = n.state

        h_value = sum(self.manhattan_distance(box, target) for box, target in (boxes, self.target))

        return h_value
    
    def manhattan_distance(self, p1, p2):
        """
        Calculate the Manhattan distance between two points.
        """
        return abs(p1[0]-p2[0]) + abs(p1[1]-p2[1])
    
    # TODO: finish the path_cost function
    # def path_cost(self, )

In [14]:
def check_action_seq(warehouse, action_seq):
    '''
    
    Determine if the sequence of actions listed in 'action_seq' is legal or not.
    
    Important notes:
      - a legal sequence of actions does not necessarily solve the puzzle.
      - an action is legal even if it pushes a box onto a taboo cell.
        
    @param warehouse: a valid Warehouse object

    @param action_seq: a sequence of legal actions.
           For example, ['Left', 'Down', Down','Right', 'Up', 'Down']
           
    @return
        The string 'Failure', if one of the action was not successul.
           For example, if the agent tries to push two boxes at the same time,
                        or push one box into a wall.
        Otherwise, if all actions were successful, return                 
               A string representing the state of the puzzle after applying
               the sequence of actions.  This must be the same string as the
               string returned by the method  Warehouse.__str__()
    '''
    
    ##         "INSERT YOUR CODE HERE"
    
    raise NotImplementedError()


In [15]:
def solve_sokoban_elem(warehouse):
    '''    
    This function should solve using elementary actions 
    the puzzle defined in a file.
    
    @param warehouse: a valid Warehouse object

    @return
        If puzzle cannot be solved return the string 'Impossible'
        If a solution was found, return a list of elementary actions that solves
            the given puzzle coded with 'Left', 'Right', 'Up', 'Down'
            For example, ['Left', 'Down', Down','Right', 'Up', 'Down']
            If the puzzle is already in a goal state, simply return []
    '''
    
    ##         "INSERT YOUR CODE HERE"
    
    raise NotImplementedError()

In [8]:
def can_go_there(warehouse, dst):
    '''    
    Determine whether the worker can walk to the cell dst=(row,column) 
    without pushing any box.
    
    @param warehouse: a valid Warehouse object

    @return
      True if the worker can walk to cell dst=(row,column) without pushing any box
      False otherwise
    '''
    
    ##         "INSERT YOUR CODE HERE"
    
    raise NotImplementedError()

In [9]:
def solve_sokoban_macro(warehouse):
    '''    
    Solve using macro actions the puzzle defined in the warehouse passed as
    a parameter. A sequence of macro actions should be 
    represented by a list M of the form
            [ ((r1,c1), a1), ((r2,c2), a2), ..., ((rn,cn), an) ]
    For example M = [ ((3,4),'Left') , ((5,2),'Up'), ((12,4),'Down') ] 
    means that the worker first goes the box at row 3 and column 4 and pushes it left,
    then goes to the box at row 5 and column 2 and pushes it up, and finally
    goes the box at row 12 and column 4 and pushes it down.
    
    @param warehouse: a valid Warehouse object

    @return
        If puzzle cannot be solved return the string 'Impossible'
        Otherwise return M a sequence of macro actions that solves the puzzle.
        If the puzzle is already in a goal state, simply return []
    '''
    
    ##         "INSERT YOUR CODE HERE"
    
    raise NotImplementedError()